In [3]:
import pandas as pd
import sys
sys.path.append('/home/greg/GitHub/')
import pyEDIutils.search as edi

## Read local status of packages

This table is where we assign core, signature, local keywords, and other categorizations

In [4]:
dtype_dic= { 'studyid': str }
#localstatus = pd.read_csv('edi_package_localstatus.csv', converters={'studyid': lambda x: str(x)})
localstatus = pd.read_csv('edi_package_localstatus.csv', dtype = dtype_dic)
#localstatus = pd.read_csv('edi_package_localstatus.csv')
print(len(localstatus))
print(localstatus.info())
localstatus.head()

211
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   studyid            211 non-null    object 
 1   pkgid              211 non-null    int64  
 2   pubdate            211 non-null    int64  
 3   title              211 non-null    object 
 4   authors            211 non-null    object 
 5   old_core           61 non-null     object 
 6   core_sig_2020prop  168 non-null    object 
 7   top10_2020prop     18 non-null     float64
 8   top10doi_2020prop  18 non-null     object 
 9   corearea           211 non-null    object 
dtypes: float64(1), int64(2), object(7)
memory usage: 16.6+ KB
None


,studyid,pkgid,pubdate,title,authors,old_core,core_sig_2020prop,top10_2020prop,top10doi_2020prop,corearea
0,1,100,2019,High resolution shrub cover raster maps of the...,"Ji, Wenjie; Hanan, Niall P.;",NaN,signature,NaN,NaN,Land Use and Land Cover Change
1,001,210001001,2011,Hydrology natural runoff plots plant cover,"Ward, Tim;",NaN,NaN,NaN,NaN,Inorganic matter cycling
2,001,210001002,2019,Rainfall runoff and sediment deposition from 2...,"Ward, Tim;",core,core,NaN,NaN,Inorganic matter cycling
3,001,210001003,2011,Hydrology plot surface runoff water chemistry,"Ward, Tim;",NaN,NaN,NaN,NaN,Inorganic matter cycling
4,002,210002001,2019,Graduated rain gauge (GRG) precipitation obser...,"Huenneke, Laura; Anderson, John;",core,core,1.0,https://doi.org/10.1007/s00442-013-2837-y,Disturbance


### Make changes if needed and resave

Add keywords, alter core/signature, etc

In [5]:
print(localstatus.studyid.unique())
sum(localstatus.studyid.isna())

['1' '001' '002' '007' '008' '010' '011' '012' '013' '015' '086' '119'
 '120' '121' '122' '126' '127' '128' '228' '262' '278' '287' '288' '301'
 '308' '327' '328' '338' '351' '365' '368' '369' '370' '379' '380' '385'
 '386' '392' '394' '395' '407' '409' '412' '425' '437' '461' '472' '520']


0

In [6]:
#Assign core areas by study id
localstatus['corearea'] = 'None'
lulc = localstatus.studyid.isin(['1','119','120','392','472'])
localstatus.loc[lulc, 'corearea'] = 'Land Use and Land Cover Change'

inorganic = localstatus.studyid.isin(['001','012','013','128','228','288','338','365','368','386',
                                      '394','395'])
localstatus.loc[inorganic, 'corearea'] = 'Inorganic matter cycling'

organic = localstatus.studyid.isin(['301','327'])
localstatus.loc[organic, 'corearea'] = 'Organic matter cycling'

pops = localstatus.studyid.isin(['007','008','010','351','370','369','385','412','520'])
localstatus.loc[pops, 'corearea'] = 'Populations'

disturbance = localstatus.studyid.isin(['002','122','126','127','278','308','328','379','380','407',
                                        '425','437','461'])
localstatus.loc[disturbance, 'corearea'] = 'Disturbance'

prod = localstatus.studyid.isin(['011','287','409'])
localstatus.loc[prod, 'corearea'] = 'Primary production'

In [7]:
# Assign core areas by pkgid (studies 015, 086, 121, 262)
#lulc = localstatus.pkgid.isna()
#localstatus.loc[lulc, 'corearea'] = 'Land Use and Land Cover Change'

inorganic = localstatus.pkgid.isin(['210015002','210015003'])
localstatus.loc[inorganic, 'corearea'] = 'Inorganic matter cycling'

organic = localstatus.pkgid.isin(['210086004'])
localstatus.loc[organic, 'corearea'] = 'Organic matter cycling'

pops = localstatus.pkgid.isin(['210086001','210086002','210086003','210086005','210086006','210086008','210086009',
                      '210121001','210121006','210262001','210262008','210262010'])
localstatus.loc[pops, 'corearea'] = 'Populations'

disturbance = localstatus.pkgid.isin(['210086007','210121005','210121008'])
localstatus.loc[disturbance, 'corearea'] = 'Disturbance'

prod = localstatus.pkgid.isin(['210015001','210262004','210262005'])
localstatus.loc[prod, 'corearea'] = 'Primary production'

In [8]:
localstatus.head()

,studyid,pkgid,pubdate,title,authors,old_core,core_sig_2020prop,top10_2020prop,top10doi_2020prop,corearea
0,1,100,2019,High resolution shrub cover raster maps of the...,"Ji, Wenjie; Hanan, Niall P.;",NaN,signature,NaN,NaN,Land Use and Land Cover Change
1,001,210001001,2011,Hydrology natural runoff plots plant cover,"Ward, Tim;",NaN,NaN,NaN,NaN,Inorganic matter cycling
2,001,210001002,2019,Rainfall runoff and sediment deposition from 2...,"Ward, Tim;",core,core,NaN,NaN,Inorganic matter cycling
3,001,210001003,2011,Hydrology plot surface runoff water chemistry,"Ward, Tim;",NaN,NaN,NaN,NaN,Inorganic matter cycling
4,002,210002001,2019,Graduated rain gauge (GRG) precipitation obser...,"Huenneke, Laura; Anderson, John;",core,core,1.0,https://doi.org/10.1007/s00442-013-2837-y,Disturbance


In [9]:
localstatus.loc[localstatus.corearea=='None',:]

,studyid,pkgid,pubdate,title,authors,old_core,core_sig_2020prop,top10_2020prop,top10doi_2020prop,corearea
18,015,210015001,2011,Transect Vegetation Biomass,"Anderson, John;",NaN,NaN,NaN,NaN,None
19,015,210015002,2011,Transect Soil Mineralization Potential (Field),"Anderson, John;",NaN,NaN,NaN,NaN,None
20,015,210015003,2011,Transect Soil Mineralization Potential (Initial),"Anderson, John;",NaN,NaN,NaN,NaN,None
21,086,210086001,2019,Vegetation cover data from line-intercept tran...,"Lightfoot, David; Bestelmeyer, Brandon;",core,core,NaN,NaN,None
22,086,210086002,2019,Quadrat vegetation cover data from the long-te...,"Lightfoot, David; Bestelmeyer, Brandon;",core,core,NaN,NaN,None
23,086,210086003,2019,Cryptogam crust data from the long-term Small ...,"Lightfoot, David; Bestelmeyer, Brandon;",core,core,NaN,NaN,None
24,086,210086004,2019,Leaf litter cover data on 1m x 1m plots from t...,"Lightfoot, David; Bestelmeyer, Brandon;",core,core,NaN,NaN,None
25,086,210086005,2019,Rabbit feces counts on 1m x 1m plots from the ...,"Lightfoot, David; Bestelmeyer, Brandon;",core,core,NaN,NaN,None
26,086,210086006,2019,Rabbit survey data on creosotebush and grassla...,"Bestelmeyer, Brandon; Lightfoot, David; School...",core,core,NaN,NaN,None
27,086,210086007,2019,Soil disturbance cover data on 1m x 1m plots f...,"Lightfoot, David; Bestelmeyer, Brandon;",core,core,NaN,NaN,None


In [10]:
localstatus_out = localstatus.copy()
#localstatus_out.to_csv('edi_package_localstatus.csv', index=False)

## Get table from EDI

This lists all data packages at EDI under the knb-lter-jrn scope.

In [63]:
import importlib
importlib.reload(edi)
importlib.reload(edi.rq)
df = edi.request_search(fields=['packageid','title','pubdate','keyword','author',
                                   'begindate','enddate','doi'])
df.head()

https://pasta.lternet.edu/package/search/eml?defType=edismax&q=%2A&fq=scope%3Aknb-lter-jrn&fl=packageid%2Ctitle%2Cpubdate%2Ckeyword%2Cauthor%2Cbegindate%2Cenddate%2Cdoi&sort=packageid%2Casc&rows=500


,packageid,title,pubdate,keywords,authors,begindate,enddate,doi
0,knb-lter-jrn.100.3,High resolution shrub cover raster maps of the...,2020,canopy cover;land surface properties;plant cov...,"Ji, Wenjie;Hanan, Niall P.",2011-01-01,2011-12-31,doi:10.6073/pasta/313fec8669bc7b4d8debf7393dd2...
1,knb-lter-jrn.210001001.62,Plant cover on 2 x 2 meter rainfall runoff plo...,2020,termites;runoff;plant cover;deserts;grasslands...,"Ward, Tim;Bolton, Susan;Schlesinger, William",1989-10-27,1990-10-19,doi:10.6073/pasta/d320f815d3183ea09fa2f18e0030...
2,knb-lter-jrn.210001002.76,Rainfall runoff and sediment deposition from 2...,2019,land cover;runoff;plant communities;deserts;gr...,"Ward, Tim",1982-09-09,1994-10-15,doi:10.6073/pasta/a6b6175f8a064fe2bd35a62e2de0...
3,knb-lter-jrn.210001003.79,Rainfall runoff water chemistry from 2 x 2 met...,2020,termites;runoff;chemical properties;deserts;gr...,"Ward, Tim;Bolton, Susan;Schlesinger, William",1988-06-27,1990-09-11,doi:10.6073/pasta/db814c25fb4bf8f6bdd7addbbcbd...
4,knb-lter-jrn.210002001.127,Graduated rain gauge (GRG) precipitation obser...,2019,precipitation;climate;rain;hydrological proces...,"Huenneke, Laura;Anderson, John",1989-01-03,2019-04-16,doi:10.6073/pasta/39920e433f27d19f0c196d431a4a...


In [49]:
df[df.packageid=='knb-lter-jrn.100.3']

,packageid,title,pubdate,keywords,authors,begindate,enddate,doi
0,knb-lter-jrn.100.3,High resolution shrub cover raster maps of the...,2020,canopy cover;land surface properties;plant cov...,"Ji, Wenjie;Hanan, Niall P.",2011-01-01,2011-12-31,doi:10.6073/pasta/313fec8669bc7b4d8debf7393dd2...


In [50]:
pkgid = df.packageid.str.split('.', expand=True).loc[:,1]
df['pkgid'] = pkgid.astype(int)
df['studyid'] = pkgid.astype(str).str[0:-2]
df.loc[pkgid.astype(int) > 210000000, 'studyid'] = pkgid.loc[pkgid.astype(int) > 210000000].str[-6:-3]
df.head()

,packageid,title,pubdate,keywords,authors,begindate,enddate,doi,pkgid,studyid
0,knb-lter-jrn.100.3,High resolution shrub cover raster maps of the...,2020,canopy cover;land surface properties;plant cov...,"Ji, Wenjie;Hanan, Niall P.",2011-01-01,2011-12-31,doi:10.6073/pasta/313fec8669bc7b4d8debf7393dd2...,100,1
1,knb-lter-jrn.210001001.62,Plant cover on 2 x 2 meter rainfall runoff plo...,2020,termites;runoff;plant cover;deserts;grasslands...,"Ward, Tim;Bolton, Susan;Schlesinger, William",1989-10-27,1990-10-19,doi:10.6073/pasta/d320f815d3183ea09fa2f18e0030...,210001001,001
2,knb-lter-jrn.210001002.76,Rainfall runoff and sediment deposition from 2...,2019,land cover;runoff;plant communities;deserts;gr...,"Ward, Tim",1982-09-09,1994-10-15,doi:10.6073/pasta/a6b6175f8a064fe2bd35a62e2de0...,210001002,001
3,knb-lter-jrn.210001003.79,Rainfall runoff water chemistry from 2 x 2 met...,2020,termites;runoff;chemical properties;deserts;gr...,"Ward, Tim;Bolton, Susan;Schlesinger, William",1988-06-27,1990-09-11,doi:10.6073/pasta/db814c25fb4bf8f6bdd7addbbcbd...,210001003,001
4,knb-lter-jrn.210002001.127,Graduated rain gauge (GRG) precipitation obser...,2019,precipitation;climate;rain;hydrological proces...,"Huenneke, Laura;Anderson, John",1989-01-03,2019-04-16,doi:10.6073/pasta/39920e433f27d19f0c196d431a4a...,210002001,002


## Make Table1


In [51]:
# Merge in core designations and rename them to "signature"
t1 = pd.merge(df, localstatus.loc[:,['pkgid','core_sig_2020prop','corearea',
                                     'top10_2020prop']], on='pkgid')
t1.core_sig_2020prop.replace('core', 'signature',inplace=True)
print(len(t1))

# Replace or add whitespace for a couple columns
t1.title = t1.title.str.replace('\n +', ' ')
t1.authors = t1.authors.str.replace(';', '; ')
# Trim version # from package id
t1.packageid = t1.packageid.map(lambda x: x.rsplit('.',1)[0])
# Add a date range
t1['drange'] = t1.begindate + ' to ' + t1.enddate
# Replace core areas with abbreviations
t1.corearea.replace('Disturbance', 'D', inplace=True)
t1.corearea.replace('Land Use and Land Cover Change', 'Lu', inplace=True)
t1.corearea.replace('Primary production', 'Pp', inplace=True)
t1.corearea.replace('Populations', 'Po', inplace=True)
t1.corearea.replace('Inorganic matter cycling', 'Im', inplace=True)
t1.corearea.replace('Organic matter cycling', 'Om', inplace=True)
# Add sorting order column for corearea
t1['corearea_sort'] = 0
t1.corearea_sort.loc[t1.corearea=='Pp'] = 1
t1.corearea_sort.loc[t1.corearea=='Om'] = 2
t1.corearea_sort.loc[t1.corearea=='D'] = 3
t1.corearea_sort.loc[t1.corearea=='Im'] = 4
t1.corearea_sort.loc[t1.corearea=='Po'] = 5
t1.corearea_sort.loc[t1.corearea=='Lu'] = 6
# Sort values
t1 = t1.sort_values(['core_sig_2020prop', 'corearea_sort', 'pkgid'],
                   ascending=['True', 'True', 'True'])

# Fill in NA values with empty string
t1['core_sig_2020prop'].fillna('', inplace=True)
t1['top10_2020prop'].fillna('', inplace=True)

t1.index = t1.loc[:,'pkgid']
t1.head()

211


/home/greg/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/home/greg/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,packageid,title,pubdate,keywords,authors,begindate,enddate,doi,pkgid,studyid,core_sig_2020prop,corearea,top10_2020prop,drange,corearea_sort
pkgid,,,,,,,,,,,,,,,
210086001,knb-lter-jrn.210086001,Vegetation cover data from line-intercept tran...,2019,land cover;plant communities;deserts;grassland...,"Lightfoot, David; Bestelmeyer, Brandon",1995-11-08,2005-12-01,doi:10.6073/pasta/44461e6afc203ec7e4e56f222e12...,210086001,086,signature,None,,1995-11-08 to 2005-12-01,0
210086002,knb-lter-jrn.210086002,Quadrat vegetation cover data from the long-te...,2019,land cover;plant communities;deserts;grassland...,"Lightfoot, David; Bestelmeyer, Brandon",1995-04-03,2015-10-29,doi:10.6073/pasta/b78fb0f8ffaa7595256d2787263d...,210086002,086,signature,None,,1995-04-03 to 2015-10-29,0
210086003,knb-lter-jrn.210086003,Cryptogam crust data from the long-term Small ...,2019,land cover;land surface properties;soil;herbiv...,"Lightfoot, David; Bestelmeyer, Brandon",1995-04-03,2005-10-28,doi:10.6073/pasta/5f3799654c06c3d1d7bea052ac2c...,210086003,086,signature,None,,1995-04-03 to 2005-10-28,0
210086004,knb-lter-jrn.210086004,Leaf litter cover data on 1m x 1m plots from t...,2019,land cover;leaf litter;deserts;grasslands;shru...,"Lightfoot, David; Bestelmeyer, Brandon",1995-04-03,2015-10-29,doi:10.6073/pasta/b22a6bdd36f6490f6732b18c2fa1...,210086004,086,signature,None,,1995-04-03 to 2015-10-29,0
210086005,knb-lter-jrn.210086005,Rabbit feces counts on 1m x 1m plots from the ...,2019,land cover;vegetation;herbivory;deserts;grassl...,"Lightfoot, David; Bestelmeyer, Brandon",1995-04-03,2015-10-29,doi:10.6073/pasta/f1a7db45fc90d318a66040c16bbe...,210086005,086,signature,None,,1995-04-03 to 2015-10-29,0


## Special cases (on Deb's orders)

In [52]:
t1.authors.loc[t1.pkgid.isin([210011001, 210011002, 210011003, 210011004, 210287001])] = 'Peters, Debra; Huenneke, Laura'
t1.authors.loc[t1.pkgid.isin([210002001])] = 'Peters, Debra; Anderson, John; Huenneke, Laura'
t1.authors.loc[t1.pkgid.isin([210121005])] = 'Peters, Debra; Anderson, John; Rango, Al; Huenneke, Laura'
t1.authors.loc[t1.pkgid.isin([210308005])] = 'Peters, Debra; Herrick, Jeff; Okin, Gregory S'
t1.authors.loc[t1.pkgid.isin([210351003])] = 'Bestelmeyer, Brandon; Yao, Jin'
# Append symbol to SMES and similar packages (multiyear=asterisk or complete=dagger)
multiyr = [210086002,210086004,210086005,210086007,210120001,210120002]
t1.title.loc[t1.pkgid.isin(multiyr)] = t1.title.loc[t1.pkgid.isin(multiyr)] + '*'
#complete = [210086001,210086003,210086008,210086009]
#t1.title.loc[t1.pkgid.isin(complete)] = t1.title.loc[t1.pkgid.isin(complete)] + u'\u2020'

# Just drop the most useful NPP package from the table (Sigh)
# and the deprecated soil water
t1 = t1.drop([210011001,210013002])
t1.head()

/home/greg/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,packageid,title,pubdate,keywords,authors,begindate,enddate,doi,pkgid,studyid,core_sig_2020prop,corearea,top10_2020prop,drange,corearea_sort
pkgid,,,,,,,,,,,,,,,
210086001,knb-lter-jrn.210086001,Vegetation cover data from line-intercept tran...,2019,land cover;plant communities;deserts;grassland...,"Lightfoot, David; Bestelmeyer, Brandon",1995-11-08,2005-12-01,doi:10.6073/pasta/44461e6afc203ec7e4e56f222e12...,210086001,086,signature,None,,1995-11-08 to 2005-12-01,0
210086002,knb-lter-jrn.210086002,Quadrat vegetation cover data from the long-te...,2019,land cover;plant communities;deserts;grassland...,"Lightfoot, David; Bestelmeyer, Brandon",1995-04-03,2015-10-29,doi:10.6073/pasta/b78fb0f8ffaa7595256d2787263d...,210086002,086,signature,None,,1995-04-03 to 2015-10-29,0
210086003,knb-lter-jrn.210086003,Cryptogam crust data from the long-term Small ...,2019,land cover;land surface properties;soil;herbiv...,"Lightfoot, David; Bestelmeyer, Brandon",1995-04-03,2005-10-28,doi:10.6073/pasta/5f3799654c06c3d1d7bea052ac2c...,210086003,086,signature,None,,1995-04-03 to 2005-10-28,0
210086004,knb-lter-jrn.210086004,Leaf litter cover data on 1m x 1m plots from t...,2019,land cover;leaf litter;deserts;grasslands;shru...,"Lightfoot, David; Bestelmeyer, Brandon",1995-04-03,2015-10-29,doi:10.6073/pasta/b22a6bdd36f6490f6732b18c2fa1...,210086004,086,signature,None,,1995-04-03 to 2015-10-29,0
210086005,knb-lter-jrn.210086005,Rabbit feces counts on 1m x 1m plots from the ...,2019,land cover;vegetation;herbivory;deserts;grassl...,"Lightfoot, David; Bestelmeyer, Brandon",1995-04-03,2015-10-29,doi:10.6073/pasta/f1a7db45fc90d318a66040c16bbe...,210086005,086,signature,None,,1995-04-03 to 2015-10-29,0


In [53]:
t1_out = t1.loc[:, ['authors','pubdate','title','packageid','corearea','drange',
                    'core_sig_2020prop','top10_2020prop']]
t1_out.to_excel('tables_out/Table1_draft.xlsx', index=False)